<a href="https://colab.research.google.com/github/DmitryIo/education/blob/master/weight_init_amp_optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Как инициализировать веса?
Сделаем мини исследование

In [0]:
import torch
import numpy as np

**Normal distribution:**

![Normal distribution](https://i.imgur.com/x2SyLi6.png)

In [0]:
стандартизация – (x - x.mean()) / x.std()
– приводим данные к виду стандартного нормального распр (mu = 0, sigma^2 = 1)

In [0]:
var – дисперсия –  sigma^2
std – среднеквадратичное отклонение – sigma

# y = WX

In [0]:
# vanishing, exploiding values -> gradients

In [0]:
x -> layer_1 -> layer_2 -> .. -> layer_100

In [0]:
x = torch.randn(128)
for i in range(100):
    w = torch.randn(128, 128) * (1 / np.sqrt(128))
    x = w @ x

In [0]:
print(x.mean().item())
print(x.std().item())

0.008587132208049297
0.7051880955696106


Хотим, чтобы x.mean() стремилось к 0, а std стремилось к 1

In [0]:
m = []
v = []
for i in range(1000):
    x = torch.randn(128)
    w = torch.randn(128, 128)* np.sqrt(2 / 128) # такая инициализация позволяет уменьшить проблему взрывающихся градиентов; torch.randn(128,128) * np.sqrt(hyp / n_size), где hyp гиперпараметр(обычно от 2 до 5), а n_size - размерность 
    y = torch.relu(x @ w)
    m.append(y.mean().item())
    v.append(y.pow(2).mean().item() - y.mean().item()**2)

In [0]:
true_mean = np.mean(m)

true_var = np.mean(v)

In [0]:
print(true_mean)
print(np.sqrt(true_var))

0.5584457634091378
0.8173048354267037


In [0]:
x = torch.randn(512)
# +-sqrt(6 / n + m), n - количество входных фич, m – выходных – glorot, xavier
# randn(0, 1) * sqrt(2 / n) – инициализация Каиминга
 
for i in range(100):
    w = torch.Tensor(512, 512).uniform_(-1, 1) * np.sqrt(5. / 512)
    x = torch.relu(w @ x)

In [0]:
x.mean(), x.std()

(tensor(3.0017e-05), tensor(4.6248e-05))

# Градиентный спуск
$$\theta_t = \theta_{t-1} - \lambda\frac{\partial{L(X, Y, \theta_{t-1})}}{\partial{\theta}}$$

# Стохастический градиентный спуск

$$\theta_t = \theta_{t-1} - \lambda\frac{\partial{L(x, y, \theta_{t-1})}}{\partial{\theta}}$$

В чем отличие от GD?
Преимущества и недостатки относительно GD?

# Градиентный спуск по батчам
$$\theta_t = \theta_{t-1} - \lambda\frac{\partial{L(X_b, Y_b, \theta_{t-1})}}{\partial{\theta}}$$

# Моментум
$$V_t = \gamma V_{t-1} + \lambda\frac{\partial{L(x, y, \theta_{t-1})}}{\partial{\theta}}$$


$$\theta_t = \theta_{t-1} - V_t$$

Какие недостатки и преимущества?

In [0]:
Nesterov Accelerated Gradient

$$V(t) = \gamma V_{t-1} + \lambda\frac{\partial{L(x, y, \theta_{t-1} - \gamma V_{t-1})}}{\partial{\theta}}$$

# Adagrad
$$g_{t, i} = \frac{\partial{L(x, y, \theta})}{\partial{\theta_{t, i}}}$$ – градиент функции ошибки по параметру $\theta_i$


$$\theta_{t+1, i} = \theta_{t, i} - \frac{\lambda}{\sqrt{G_{t, ii} + \epsilon}} * g_{t, i}$$

$$G_{t, ii} = G_{t-1, ii} + g_{t, i}^2$$

$G_{t}$ – диагональная матрица, каждый элемент которой – сумма квадратов градиентов соотв параметра i за предыдущие шаги. 

$G_{t, 11}$

# RMSProp

$$G_{t, ii} = \alpha G_{t-1, ii} + (1 - \alpha)g_{t, i}^2$$, alpha = 0.9

$$\theta_{t+1, i} = \theta_{t, i} - \frac{\lambda}{\sqrt{G_{t, ii} + \epsilon}} * g_{t, i}$$



# Adadelta

$$G_{t, ii} = \alpha G_{t-1, ii} + (1 - \alpha)g_{t, i}^2$$

$$\mathbb{E}(\Delta\theta^2)_{t+1} = \alpha\mathbb{E}(\Delta\theta^2)_{t} + (1- \alpha)\Delta\theta^2$$

$$\theta_{t+1, i} = \theta_{t, i} - \frac{\sqrt{\mathbb{E}(\Delta\theta^2)_t + \epsilon}}{\sqrt{G_{t, ii} + \epsilon}} * g_{t, i}$$



# Adam
$$g_{t, i} = \frac{\partial{L(x, y, \theta})}{\partial{\theta_{t, i}}}$$ – градиент функции ошибки по параметру $\theta_i$

$$m_t = \beta_{1}m_{t-1} + (1-\beta_1)g_t$$
$$v_t = \beta_{2}v_{t-1} + (1 - \beta_2)g_t^2$$

$$\theta_{t+1} = \theta_{t} - \frac{\lambda}{\sqrt{v_t + \epsilon}}m_t$$ $\beta_1 = 0.9, \beta_2 = 0.999, \epsilon=10^{-8}$